<a href="https://colab.research.google.com/github/financieras/campus/blob/main/pilas/pilas20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Push Swap método "Modern Times"
En honor a la película de 1936 de Chaplin.

In [1]:
# FUNCIONES

def lista_ordenada(lis, orden_inverso=False): # la función da True si ponemos lista_ordenada([4,3,2,1], orden_inverso=True)
    ordenada = False
    aux = lis[:]
    aux.sort(reverse=orden_inverso)
    if aux == lis:
        ordenada = True
    return ordenada

def sa(a,b):
    if len(a) > 1: a[0],a[1] = a[1],a[0]
    return a,b
def sb(a,b):
    if len(b) > 1: b[0],b[1] = b[1],b[0]
    return a,b
def ss(a,b):
    sa(a,b)
    sb(a,b)
    return a,b
def pa(a,b):
    if len(b) > 0:
        a.insert(0, b[0])
        b.pop(0)
    return a,b
def pb(a,b):
    if len(a) > 0:
        b.insert(0, a[0])
        a.pop(0)
    return a,b
def ra(a,b):
    if len(a) > 1: a.append(a.pop(0))
    return a,b
def rb(a,b):
    if len(b) > 1: b.append(b.pop(0))
    return a,b
def rr(a,b):
    ra(a,b)
    rb(a,b)
    return a,b
def rra(a,b):
    if len(a) > 1: a.insert(0, a.pop())
    return a,b
def rrb(a,b):
    if len(b) > 1: b.insert(0, b.pop())
    return a,b
def rrr(a,b):
    rra(a,b)
    rrb(a,b)
    return a,b

In [2]:
# Generación de la pila a
from random import sample, seed
seed()
n = 100   # número de elementos de la pila
#a = sample(range(1, n+1), n)
a = [35, 90, 6, 15, 7, 40, 85, 63, 50, 13, 45, 8, 76, 33, 25, 27, 42, 20, 94, 48, 91, 29, 95, 77, 82, 98, 86, 69, 60, 58, 79, 55, 24, 12, 28, 93, 73, 54, 84, 44, 10, 75, 3, 32, 89, 52, 51, 16, 56, 70, 22, 11, 59, 68, 100, 96, 71, 21, 26, 38, 1, 46, 92, 36, 88, 31, 66, 34, 74, 23, 5, 9, 43, 49, 37, 80, 18, 17, 67, 83, 78, 61, 47, 57, 64, 81, 97, 53, 65, 30, 2, 99, 14, 87, 39, 19, 72, 4, 62, 41]
print('a:', a)
b = []
a_original = a[:]
n = len(a)

a: [35, 90, 6, 15, 7, 40, 85, 63, 50, 13, 45, 8, 76, 33, 25, 27, 42, 20, 94, 48, 91, 29, 95, 77, 82, 98, 86, 69, 60, 58, 79, 55, 24, 12, 28, 93, 73, 54, 84, 44, 10, 75, 3, 32, 89, 52, 51, 16, 56, 70, 22, 11, 59, 68, 100, 96, 71, 21, 26, 38, 1, 46, 92, 36, 88, 31, 66, 34, 74, 23, 5, 9, 43, 49, 37, 80, 18, 17, 67, 83, 78, 61, 47, 57, 64, 81, 97, 53, 65, 30, 2, 99, 14, 87, 39, 19, 72, 4, 62, 41]


In [3]:
def algoritmo_LIS(arr):
    n = len(arr)
    lis = [1]*n
    prev = list(range(n))  # una lista con los index
    
    # Compute optimized LIS values in bottom up manner
    for i in range (1, n):
        for j in range(i):
            if arr[i] > arr[j] and lis[i] < lis[j] + 1:
                lis[i] = lis[j]+1
                prev[i] = j

    # Initialize maximum to 0 to get the maximum of all
    # LIS
    maximum = 0
    idx = 0

    # Pick maximum of all LIS values
    for i in range(n):
        if maximum < lis[i]:
            maximum = lis[i]
            idx = i

    seq = [arr[idx]]
    while idx != prev[idx]:
        idx = prev[idx]
        seq.append(arr[idx])

    return (maximum, list(reversed(seq)))

(nlis, lis) = algoritmo_LIS(a)   #nlist es el número de elementos de la lista lis

In [4]:
# Llevamos de la pila A a la B los elementos de la LIS
contador = 0
for i in range(nlis):
    while a_original[contador] != lis[i]:
        ra(a,b); contador += 1; 
    else:
        pb(a,b); contador += 1
print(a_original)
print(lis)
print(a)
print(b)
print("contador:", contador)

[35, 90, 6, 15, 7, 40, 85, 63, 50, 13, 45, 8, 76, 33, 25, 27, 42, 20, 94, 48, 91, 29, 95, 77, 82, 98, 86, 69, 60, 58, 79, 55, 24, 12, 28, 93, 73, 54, 84, 44, 10, 75, 3, 32, 89, 52, 51, 16, 56, 70, 22, 11, 59, 68, 100, 96, 71, 21, 26, 38, 1, 46, 92, 36, 88, 31, 66, 34, 74, 23, 5, 9, 43, 49, 37, 80, 18, 17, 67, 83, 78, 61, 47, 57, 64, 81, 97, 53, 65, 30, 2, 99, 14, 87, 39, 19, 72, 4, 62, 41]
[6, 7, 13, 25, 27, 42, 48, 55, 56, 59, 68, 71, 74, 80, 83, 97, 99]
[14, 87, 39, 19, 72, 4, 62, 41, 35, 90, 15, 40, 85, 63, 50, 45, 8, 76, 33, 20, 94, 91, 29, 95, 77, 82, 98, 86, 69, 60, 58, 79, 24, 12, 28, 93, 73, 54, 84, 44, 10, 75, 3, 32, 89, 52, 51, 16, 70, 22, 11, 100, 96, 21, 26, 38, 1, 46, 92, 36, 88, 31, 66, 34, 23, 5, 9, 43, 49, 37, 18, 17, 67, 78, 61, 47, 57, 64, 81, 53, 65, 30, 2]
[99, 97, 83, 80, 74, 71, 68, 59, 56, 55, 48, 42, 27, 25, 13, 7, 6]
contador: 92


In [ ]:
def pasos_estimados(a, b, index_a):
    # estima el número de pasos para colocar el elemento de la pila A cuyo index_a nos pasan, en el sitio adecuado de la pila B
    # las pilas A y B que recibimos como parámetros no se han de alterar por parte de esta función
    


    conta2 = 0
    while not lista_ordenada(a):   # se repite hasta que A quede ordenada. Si solo queda un elemento en A es que está ordenada
        while min(a) != a[0]:   # buscamos el mínimo y vamos haciendo ra o rra hasta que quede el primero
            if a.index(min(a)) <= int(len(b)/2):
                ra(a,b)
            else:
                rra(a,b)
            conta2 += 1
        if not lista_ordenada(a):
            pb(a,b); conta2 += 1






    




    return pasos


def busca_candidato(a, b):
    candidato = 0         # determinar el elemento de la pila A que menos pasos necesita para ubicarse en su sitio de la pila B
    pasos_min = 2**31-1   # inicializamos la variable con infinito o un número de pasos que sabemos que es muy muy grande 2**31-1 == 2147483647

    for index_a in range(len(a)):
        p = pasos(a, b, index_a)
        if p < pasos_min:
            pasos_min = p
            candidato = a[index_a]   #ahora el nuevo candidato es el elemento de A que requiere, de momento, menos pasos para ir a su sitio en la pila B
    return candidato




In [ ]:
elemento = 6
b = [20, 15, 12, 10, 5, 2]

dif = 2**31-1  #inicializamos la diferencia con infinito o casi
index_insercion = 0   #inicializamos el index de B donde se realizará la inserción del elemento
for i,v in enumerate(b):
    if abs(v-elemento) < dif:
        dif = abs(v-elemento)
        indice_insercion = i

print(indice_insercion)

# convertirmos el indice_insercion en pasos necesarios para el elemento considerado

mitad = int(len(b)/2)

print("mitad:", mitad)

if elemento < b[mitad]:
    print("signo negativo")
else:
    print("signo positivo")


#    if a.index(min(a)) <= int(len(b)/2):




4
mitad: 3
signo negativo
